### 1. Import Libraries

In [23]:
import yfinance as yf
from scipy.optimize import minimize
import numpy as np
import pandas as pd
from itertools import combinations

### 2. Data Obtention

In [24]:
# Lista de empresas a evaluar
tickers_list = [
    'CCWFX', 'RCWBX', 'WBFFX', 'WFBFX', 'RCEBX', 'RCWHX', 'RCWEX', 'BFWFX', 'CWBCX', 'RCWCX',
    'FCWBX', 'CCWEX', 'CCWCX', 'CWBFX', 'CCWAX', 'FWBCX', 'RCWGX', 'RCWFX', 'RCWAX', 'CRDOX',
    'GIOCX', 'GIOSX', 'GIOIX', 'GIOAX', 'GIOPX', 'MEDGX', 'MEDAX', 'MEDEX', 'MEDFX', 'MEDHX',
    'MEDBX', 'MEDCX', 'MEDIX', 'MEDDX', 'TRKZX', 'PRHIX', 'PAHIX', 'PRHYX', 'VMIAX', 'BBMHX',
    'PTYIX', 'PATFX', 'PRFHX', 'FEGOX', 'FEGIX', 'FEURX', 'SGGDX', 'ERABX', 'EBABX', 'EIBAX',
    'ECBAX', 'PFRIX', 'FRFCX', 'FRFZX', 'FRFAX', '0P0000TISB', '0P0000TISC', '0P0001HZWL', '0P0001HZWM',
    'OGMYX', 'OGMIX', 'OGMNX', 'IOGYX', 'OGMCX', 'OPGSX', 'MHICX', 'MHIBX', 'MIHRX', 'MHIGX',
    'MHIHX', 'MHIJX', 'MHIIX', 'MHITX', 'MHIKX', 'TRHYX', 'FGDTX', 'FGDCX', 'FIJDX', 'FGDIX',
    'FGDAX', 'FSAGX', 'ISHSX', 'ISHYX', 'ISHFX', 'ISHAX', 'ISHCX', 'FRGOX', 'FGPMX', 'FKRCX',
    'FGADX', 'CSOAX', 'CSOIX', 'CSOCX', 'SGDLX', 'SGDIX', 'HWHZX', 'HWHAX', 'HWHIX', 'RHYAX',
    'RGHYX'
]
# Descargar datos históricos
tickers = yf.Tickers(" ".join(tickers_list))
hist = tickers.history(start='2021-01-01',end='2024-12-31')
adj_close = hist['Close'].dropna(axis=1, how='any')

[*********************100%***********************]  100 of 100 completed


### 3. Portfolios Analysis

In [25]:
# Función para calcular métricas del portafolio
def calcular_metricas(adj_close_values):
    R = np.log(adj_close_values[1:] / adj_close_values[:-1])  # Retornos logarítmicos
    RE = np.mean(R, axis=0) * 252  # Retorno esperado anualizado
    RI = np.std(R, axis=0) * np.sqrt(252)  # Riesgo anualizado
    S = np.cov(R, rowvar=False)  # Matriz de covarianza
    corr = np.corrcoef(R, rowvar=False)  # Matriz de correlación
    return RE, RI, S, corr

# Obtener métricas generales
adj_close_values = adj_close.values
RE, RI, S, correlation_matrix = calcular_metricas(adj_close_values)

# Evaluar todas las combinaciones posibles de 4 activos
mejores_portafolios = []
for subset in combinations(adj_close.columns, 4):
    indices = [adj_close.columns.get_loc(ticker) for ticker in subset]
    corr_submatrix = correlation_matrix[np.ix_(indices, indices)]

    # Filtrar combinaciones con correlación > 0.5
    if np.any(np.triu(corr_submatrix, k=1) > 0.5):
        continue  # Saltar esta combinación

    # Extraer datos de la combinación aceptable
    RE_sub = RE[indices]
    S_sub = S[np.ix_(indices, indices)]
    weights = np.ones(4) / 4  # Pesos iniciales iguales

    # Definir restricciones y límites
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    bounds = [(0, None)] * 4

    # Optimizar el portafolio
    res = minimize(lambda w: w @ S_sub @ w.T, x0=weights, method='SLSQP', bounds=bounds, constraints=constraints, options={'disp': False, 'maxiter': 1000, 'ftol': 1e-12})

    if res.success:
        # Calcular métricas del portafolio optimizado
        ReP = res.x @ RE_sub.T
        varP = res.x @ S_sub @ res.x.T
        RiP = np.sqrt(varP)*np.sqrt(252)
        SharpeP = ReP / RiP

        # Guardar resultados
        mejores_portafolios.append({
            "Activos": subset,
            "ReP (%)": round(ReP * 100, 4),
            "RiP (%)": round(RiP * 100, 4),
            "SharpeP": round(SharpeP, 4)
        })

# Ordenar por el mejor Sharpe Ratio
mejores_portafolios = sorted(mejores_portafolios, key=lambda x: x["SharpeP"], reverse=True)

# Convertir a DataFrame y mostrar los resultados
df_resultados = pd.DataFrame(mejores_portafolios)
print(df_resultados)

# Guardar en un archivo Excel
df_resultados.to_excel("mejores_portafolios.xlsx", index=False)
print("\nResultados guardados en 'mejores_portafolios.xlsx'")

                           Activos  ReP (%)  RiP (%)  SharpeP
0     (FEURX, FRFZX, MEDGX, VMIAX)   5.8346   3.0925   1.8867
1     (FEGIX, FRFZX, MEDGX, VMIAX)   5.8338   3.0926   1.8864
2     (FRFZX, MEDGX, SGGDX, VMIAX)   5.8322   3.0925   1.8859
3     (FEURX, FRFZX, MEDHX, VMIAX)   5.8286   3.0924   1.8848
4     (FEURX, FRFZX, MEDIX, VMIAX)   5.8287   3.0924   1.8848
...                            ...      ...      ...      ...
5591  (CRDOX, ISHAX, SGGDX, VMIAX)   1.5127   2.7377   0.5526
5592  (CRDOX, FEGIX, ISHCX, VMIAX)   0.8143   2.6522   0.3070
5593  (CRDOX, FEGOX, ISHCX, VMIAX)   0.8143   2.6522   0.3070
5594  (CRDOX, FEURX, ISHCX, VMIAX)   0.8143   2.6522   0.3070
5595  (CRDOX, ISHCX, SGGDX, VMIAX)   0.8143   2.6522   0.3070

[5596 rows x 4 columns]

Resultados guardados en 'mejores_portafolios.xlsx'
